# Pruebas de la librería del TFG
Este cuaderno contiene pruebas experimentales para la librería desarrollada en
el Trabajo de Fin de Grado (TFG). Las pruebas están diseñadas para explorar
nuevas características de la librería.

## Instalación de un copia desde el repositorio

In [ ]:
!pip install -q git+https://github.com/wvenialbo/miniature-giggle.git@main

### Importación y verificación de la librería

In [ ]:
import tfg.storage
import tfg.utils

print(tfg.__version__)

## Utilidad para descarga con streaming y reporte de progreso

In [ ]:
import pathlib as pl
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def smart_download(ds, remote_uris: list[str], local_dest: str | pl.Path, max_workers: int = 4):
    """
    Descarga archivos usando streaming.
    """
    local_root = pl.Path(local_dest)

    # Ejecución
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Barra de progreso global (por archivos)
        with tqdm(total=len(remote_uris), desc="Progreso Global", unit="file", position=0, leave=True) as main_pbar:
            # Asignamos una posición a cada hilo para que las barras de tqdm se apilen
            futures = [
                executor.submit(_download_one, ds, local_root, uri, i % max_workers + 1)
                for i, uri in enumerate(remote_uris)
            ]
            for future in as_completed(futures):
                main_pbar.update(1)

def _download_one(ds, local_root, uri: str, position: int):
    # 1. Intentar obtener el tamaño para la barra de progreso
    total_size = ds.get_size(uri=uri)

    # 2. Preparar ruta local
    # (Uso de lstrip para asegurar que sea relativa al root local)
    relative_path = uri.split(ds.mountpoint)[-1].lstrip("/")
    target_path = local_root / relative_path
    target_path.parent.mkdir(parents=True, exist_ok=True)

    # 3. Descarga con streaming
    with tqdm(
        total=total_size,
        unit='B',
        unit_scale=True,
        desc=uri.split('/')[-1],
        leave=False, # La barra desaparece al terminar para no saturar
        position=position # Evita que las barras se solapen en multihilo
    ) as pbar:
        with open(target_path, "wb") as f:
            for chunk in ds.stream(uri=uri):
                f.write(chunk)
                pbar.update(len(chunk))

## Mock de un Datasource con streaming

In [ ]:
import random
import time
import typing as tp
from dataclasses import dataclass

@dataclass
class MockResponse:
    headers: dict
    def raise_for_status(self): pass

class MockMapper:

    def to_native(self, uri: str) -> str:
        return uri

class BackendMock:
    def __init__(self):
        # Simulamos que tenemos un archivo de 50MB
        self.file_size = 50 * 1024 * 1024
        self.chunk_count = 50 # Lo dividiremos en 50 partes de 1MB

    def size(self, *, uri: str) -> int:
        return self.file_size

    def read_chunk(self, *, uri: str, chunk_size: int = 1024 * 1024) -> tp.Iterable[bytes]:
        for _ in range(self.chunk_count):
            latencia = random.uniform(0.01, 0.15)
            time.sleep(latencia)  # Simulamos latencia de red
            yield b"0" * chunk_size # Enviamos un MB de "datos" dummy

# Mock simple del Datasource para que funcione con smart_download
class DatasourceMock:
    def __init__(self):
        self.backend = BackendMock()
        self.mapper = MockMapper()
        self.mountpoint = "/ncei_data"

    def get_size(self, uri: str) -> int:
        return self.backend.size(uri=uri)

    def stream(self, uri: str) -> tp.Iterable[bytes]:
        return self.backend.read_chunk(uri=uri)

## Caso de prueba con Mock

In [ ]:
from tqdm.notebook import tqdm
import pathlib as pl

# 1. Instanciar el mock
ds_mock = DatasourceMock()

# 2. Definir archivos ficticios para descargar
archivos_ficticios = [
    "/ncei_data/sensor_v1_2024.nc",
    "/ncei_data/sensor_v2_2024.nc",
    "/ncei_data/sensor_v3_2024.nc",
    "/ncei_data/sensor_v4_2024.nc",
    "/ncei_data/sensor_v5_2024.nc",
    "/ncei_data/metadata_global.nc",
]

# 3. Ejecutar la descarga inteligente
print("Iniciando prueba de descarga concurrente con streaming...")
smart_download(
    ds=ds_mock,
    remote_uris=archivos_ficticios,
    local_dest="./test_mock_download",
    max_workers=3
)

## Casos de prueba con datasources reales

### Servidor NCEI Archive - NOAA (HTTP)

In [ ]:
nc = tfg.storage.use_ncei_archive(dataset_path="gridsat-goes/access/goes", cache_file="ncei.json", expire_after=100.0)
ls = tfg.storage.use_local_drive(root_path=".")

content = nc.list(prefix="/2007/08/")
print(f"Total de archivos: {len(content)}")

print("Iniciando prueba de descarga concurrente con streaming...")
smart_download(
    ds=nc,
    remote_uris=content[:2*3],
    local_dest="./test_ncei_download",
    max_workers=3,
)

### Amazon Web Services - NOAA (S3)

In [ ]:
ds = tfg.storage.use_aws_cloud(bucket="noaa-goes16", cache_file="aws.json", expire_after=100.0)
ls = tfg.storage.use_local_drive(root_path=".")

content = ds.list(prefix="/ABI-L2-CMIPF/2020/319/20")
print(f"Total de archivos: {len(content)}")

print("Iniciando prueba de descarga concurrente con streaming...")
smart_download(
    ds=ds,
    remote_uris=content[:2*3],
    local_dest="./test_aws_download",
    max_workers=3,
)

### Google Cloud Storage - NOAA (GCS)

In [ ]:
gc = tfg.storage.use_gcs_cloud(bucket="gcp-public-data-goes-16", cache_file="gcs.json", expire_after=100.0)
ls = tfg.storage.use_local_drive(root_path=".")

content = ds.list(prefix="/ABI-L2-CMIPF/2020/319/20")
print(f"Total de archivos: {len(content)}")

print("Iniciando prueba de descarga concurrente con streaming...")
smart_download(
    ds=ds,
    remote_uris=content[:2*3],
    local_dest="./test_gcs_download",
    max_workers=3,
)

### Guardar archivo en el sistema local de archivos

In [ ]:
data = ds.load(uri=content[0])
ls.save(uri=content[0], data=data)
print(type(data))

### Guardar archivo en el Drive (Montado en Colab)

In [ ]:
if tfg.utils.running_on_colab():
    cd = tfg.storage.use_colab_drive()
    cd.save(uri=content[0], data=data)
    print(type(data))

### Guardar archivo en el Drive (Usando Google Drive API)

In [ ]:
gd = tfg.storage.use_google_drive()
gd.save(uri=content[0], data=data)
print(type(data))